<a href="https://colab.research.google.com/github/marce3-2140/sales-predictions/blob/main/Sales_Predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
filename = '/content/sales_predictions_2023.csv'

In [3]:
df = pd.read_csv(filename)